# Importing Libraries

In [1]:
import pickle 
import mediapipe as mp
import cv2
import pandas as pd
import numpy as np
import warnings


In [2]:
mp_drawing = mp.solutions.drawing_utils # Drawing helpers / helps with drawing landmarks
mp_holistic = mp.solutions.holistic # Mediapipe Solutions / Gives Landmark Detection Models

# Loading The Model

In [3]:
with open('Pose_estimation.pkl', 'rb') as f:
    model = pickle.load(f)

In [4]:
model

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('randomforestclassifier', RandomForestClassifier())])

# Making Detections

1. We Detect Landmarks
2. We Predict Pose based on landmark coordinates
3. Render landmarks and body languages using openCV

In [1]:
#----------------------------------------connecting to webcam--------------------------------------------------
cap = cv2.VideoCapture(0)



#------------------------------------- Initiate holistic model-------------------------------------------------
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    while cap.isOpened():          # Looping through each Frame that is coming through webcam
        
        ret, frame = cap.read()    # Keeping image from webcam and keeping it into this variable `frame`
        
        
        
        
        #---------------------- Recolor Feed of Mediapipe-------------------------------------------
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)   #converting input frame from the BGR to the RGB.
        image.flags.writeable = False                    # prevent accidental modification of image data
        
        
        results = holistic.process(image)
        # passes the preprocessed image to the process method of the holistic object.
        # The process methodreturns a results object containing the detected landmarks and other information.
        
         
        #----------------- Recolor image back to BGR for rendering-----------------------------------
        image.flags.writeable = True                     # allowing modifications to the image data.
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)   # converting input frame from the RGB to the BGR.
        
        
        #-----------------------------------------Drawing Landmarks-------------------------------
        # 1. Draw face landmarks
        mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS, 
                                 mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1),
                                 mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                                 )
        
        # 2. Right hand
        mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                                 )

        # 3. Left Hand
        mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                                 )

        # 4. Pose Detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                 )
#-------------------------------------------------------------------------------------------------------------------------
#------------------------------------------------------------------------------------------------------------------------
#------------------------------------------------------------------------------------------------------------------------
#-------------------------------------------------------------------------------------------------------------------------
#---------------------------------------------------------------------------------------------------------        
#-------------------------------------------------------------------------------------------------------------------------
#------------------------------------------------------------------------------------------------------------------------
#------------------------------------------------------------------------------------------------------------------------
#-------------------------------------------------------------------------------------------------------------------------
#------------------------------- Make Detections--------------------------------------------------------------------------
            # Extract Pose landmarks
        
        pose = results.pose_landmarks.landmark
        pose_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in pose]).flatten())
            
            # Extract Face landmarks
        face = results.face_landmarks.landmark
        face_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in face]).flatten())
            
            # Concate rows
        row = pose_row+face_row
            
    #------------------------------------------------------------------------------------------------------------------
            # DETECT
            
                       
        X = pd.DataFrame([row])
        body_language_class = model.predict(X)[0]
        body_language_prob = model.predict_proba(X)[0]
        print(body_language_class, body_language_prob)
            
        #  Get status box
        #               (Top cord, bottom cordinate, color of box), -1(fill box with color, line thickness hai)
        cv2.rectangle(image, (0,0), (250, 60), (200, 250, 16), -1)
                            
            
        #  Display Class
        cv2.putText(image, 'CLASS', (95,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
        cv2.putText(image, body_language_class.split(' ')[0]
                    , (90,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        
        # Display Probability
        cv2.putText(image, 'PROB'
                    , (15,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
        cv2.putText(image, str(round(body_language_prob[np.argmax(body_language_prob)],2))
                    , (10,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

                        
        cv2.imshow('Raw Webcam Feed', image)

        
        #--------------------------------------------------Closing WebCAM ----------------------------------------
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

NameError: name 'cv2' is not defined